# Data Science Ex 13 - Clustering (Evaluation & Advanced Methods)

17.05.2020, Lukas Kretschmar (lukas.kretschmar@hsr.ch)

## Let's have some Fun with Evaluation of Clusters and Advanced approaches!

In this exercise, we are going to have a look at how you can evaluate clusters.
And further, we introduce another clustering approach out there.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()

## Introduction

Before we head into the details of this introduction, we need some example data.

In [ ]:
data_full = pd.read_csv("./Demo_Clustering.csv")
data = data_full[["x", "y"]]
data_full.head(5)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
data_full.plot.scatter("x", "y", ax=ax, c="label", cmap="rainbow", colorbar=False)

### Evaluation of Clusters

Well, finding the correct number of clusters or just reasoning if and why clusters make sense, is a science for itself.
There are methods out there we can use to come a bit closer to a good answer, as you saw with the *elbow method*.
Here, we introduce another approach - called the **silhouette analysis**.

#### Silhouette Analysis

References: https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html & https://en.wikipedia.org/wiki/Silhouette_(clustering)

The silhouette analysis shows how distinct clusters are.
The analysis uses a concept called *silhouette value* which ranges between `-1` and `1` and shows how close a point is to the other points of its cluster.
A value close to `1` means that a point is close to the points in its cluster, but far away to the points of the next closest cluster.
The closer the value is to `0`, the closer a value is also to another cluster.
And negative values indicate that points of another cluster are actually closer than the points of its current cluster.

Mathematically speaking, we have values
- $a(i)$: mean distance of point $i$ to the points in the same cluster
- $b(i)$: mean distance of point $i$ to the points of the next closest cluster

and the formulas for calculating the silhouette value

- $s(i) = \frac{b(i) - a(i)}{max\{a(i),b(i)\}}$: if $i$ is not alone in a cluster
- $s(i) = 0$: if $i$ is the only point in its cluster

Let's have look what this would mean for our example data.

Since the silhouette analysis relies on distances between points, clustering algorithms like k-Means are better suited than others (i.e. density-based approaches).

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

To simpify the visualization, we create a helper method that handles all the drawing.

In [ ]:
from sklearn.base import clone
from matplotlib import cm

def silhouetteAnalysis(data, model, n_clusters):
    m = clone(model)                                 # Create copy of the model
    m.set_params(n_clusters=n_clusters)              # Setting the numbers of clusters
    l_pred = m.fit_predict(data)
    coef = silhouette_score(data, l_pred)            # Get the silhouette coefficient
    print(f"Silhouette coefficient for {n_clusters}: {np.round(coef, 6)}")
    values = silhouette_samples(data, l_pred)        # Get the silhouette values
    colors = cm.get_cmap("rainbow", n_clusters)      # Getting colors
    fig, ax = plt.subplots(1,2,figsize=(20,10))
    fig.suptitle(f"Silhouette Analysis with {n_clusters} Clusters")
    margin = 10
    y_bottom = margin
    # Plotting the silhouette values
    for c in range(n_clusters):
        values_c = values[l_pred == c]               # Just getting values of cluster c
        values_c = np.sort(values_c)                 # Sorting the values
        size_c = len(values_c)                       # Getting the number of values
        y_top = y_bottom + size_c                    # Calculating the value on the y-axis to draw to
        color = colors(c/n_clusters)                 # Selecting the color of the cluster
        # Plotting the values
        (ax[0].fill_betweenx(                        # Fill an area
            np.arange(y_bottom, y_top),              # Range on y-axis to fill
            0,                                       # Start value on x-axis
            values_c,                                # End value on x-axis
            fc=color, ec=color, alpha=.7))           # Some styling
        # Plotting the cluster number to the left
        ax[0].text(-.05, y_bottom + .5 * size_c, str(c))
        y_bottom = y_top + margin                    # Calculating the next start value on the y-axis
    ax[0].axvline(coef, c="k", ls="--")              # Plotting a vertical line for the silhouette coefficient
    ax[0].set(title="Silhouette Values", xlabel="Silhouette Values", ylabel="Clusters", yticks=[], xlim=(-.1,1))
    
    # Plotting the data
    centers = m.cluster_centers_
    ax[1].scatter(data["x"], data["y"], s=20, c=l_pred, cmap=colors)          # Draw points
    ax[1].scatter(centers[:,0], centers[:,1], s=200, c="k")                   # Draw centers in black
    for i, c in enumerate(centers):
        ax[1].text(c[0], c[1], f"{i}", c="white", ha="center", va="center")   # Write cluster number in white
    ax[1].set(title="Data")

And now we can simply use the method with our given clustering algorithm and the number of clusters we want to test.

In [ ]:
model = KMeans(random_state=42)
clusters = [2,3,4,5,6,7,8,9]
for c in clusters:
    silhouetteAnalysis(data, model, c)

As we can see, we get some information back from the silhouette analysis.
The scatter plot on the right is mainly for visualization so we see how the clusters were formed.
The more interesting plot is on the left.

Those "blades" represent the silhouette values per cluster.
They have these shapes since we sorted the values per cluster in the helper method above.
The longer the shape, the more condense the cluster.
And the wider a shape, the more points are in the cluster.
The vertical dashed line represents the silhouette coefficient which is the mean of all silhouette values.

The goal is now to find the best number of clusters.
And the silhouette coefficient is an indicator to get there.
We have to choose the number by the following criteria:
- Silhouette coefficient (higher is better)
- Fluctuation of silhouette values (lower is better)
- Silhouette values should be close to the silhouette coefficient (closer is better)
- Size of clusters (equally distributed can be better)

**Note:** Having wastly different sizes of clusters can also be explained by their density.
So don't assume that clusters have to be of the same size.
Depending on the data (as in the data used here), clusters can be big because they just have many points close to each other.

Based on these criteria, we can now assess the given configurations.
- **2 clusters**
 - Silhouette coefficient to low
 - Clusters to big
- **3 clusters**
 - Fluctuation to big
 - Cluster 1 is good, clusters 0 & 2 clearly not
- **4 clusters**
 - Fluctuation to big
 - Clusters 1 & 3 are quite good, 0 & 2 not
- **5 clusters**
 - Looks promising
 - But cluster 2 indicates that it's quite scattered
- **6 clusters**
 - Better than with 5
 - But cluster 5 indicates that it's still scattered
- **7 clusters**
 - Has a lower silhouette coefficient as with 6
 - But the clusters are denser
- **8 clusters**
 - Clusters 0 & 7 indicate that we have to many clusters (more values are closer to 0)
- **9 clusters**
 - Silhouette coefficient is low again
 - Several clusters have low and even negative values

So based on this analysis, we suggest that **7 clusters** can be found in this dataset.
But **6 clusters** would also be an appropriate conclusion.

#### Plotting

The seaborn packages offers some good visualizations for your data.
You'll find a [gallery of its capabilities on its site](https://seaborn.pydata.org/examples/index.html). 

##### Kernel Density Estimation (KDE)

Reference: https://seaborn.pydata.org/tutorial/distributions.html

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,10))
sns.kdeplot(data["x"], data["y"], ax=ax[0])

ax[1].scatter(data_full["x"], data_full["y"], s=5, c=data_full["label"], cmap="rainbow", alpha=.8)
sns.kdeplot(data["x"], data["y"], ax=ax[1])

If you have just 2 dimensinal data, the KDE-jointplot could also be interesting.

In [ ]:
sns.jointplot("x", "y", data=data, height=8)

In [ ]:
sns.jointplot(data=data, x="x", y="y", kind="kde", height=8)

The darker an area, the higher the density.
And you see on the sides a distribution of all the values on one axis.

We can also use a simple hex-bin visualization.

In [ ]:
sns.jointplot(data=data, x="x", y="y", kind="hex", height=8)

##### Pairwise Plot

We can also use seaborn to simply plot all combinations of features.
This can give us a good idea how the data is distributed or we can spot correlations.

In [ ]:
data = pd.read_csv("./Demo_4Features.csv")
data.head(5)

In [ ]:
sns.pairplot(data, height=5)

In [ ]:
sns.pairplot(data, hue="label", height=5)

### Advanced Methods

We have seen many different clustering approaches:
- Partitioning (k-Means & k-Medoids)
- Hierarchical (Agglomerative)
- Density-based (DENCLUE)

Below is another example of a clustering method.

In [ ]:
data_full = pd.read_csv("./Demo_Clustering.csv")
data = data_full[["x", "y"]]
data_full.head(5)

In [ ]:
from sklearn.base import clone
import math

def multi_plot(model, data, param, values, cols=2, alpha=.5):  
    rows = math.ceil(len(values)/cols)
    fig, ax = plt.subplots(rows, cols, figsize=(cols*10,rows*10))
    for i in range(0, len(values)):
        print(f"Calculating {i+1}/{len(values)} ...")
        m = clone(model)
        m.set_params(**{param:values[i]})
        l_pred = m.fit_predict(data)
        c = int(i % cols)
        r = int(i / cols)
        data_cluster = data[l_pred != -1]
        label_cluster = l_pred[l_pred != -1]
        data_outlier = data[l_pred == -1]
        ax_rc = ax[r,c] if rows > 1 else ax[c]
        data_outlier.plot.scatter("x", "y", ax=ax_rc, c="k", alpha=alpha)
        data_cluster.plot.scatter("x", "y", ax=ax_rc, c=label_cluster, cmap="rainbow", colorbar=False)
        ax_rc.set(title=f"{m}")
    if len(values)%2:
        if(rows > 1):
            ax[rows-1, cols-1].set_axis_off()
        else:
            ax[cols-1].set_axis_off()
    print("Rendering...")

#### DBSCAN

Reference: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

DBSCAN (**D**ensity-**B**ased **S**patial **C**lustering of **A**pplications with **N**oise) is a well known density-based clustering algorithm.
It tries to find cores (with high density) and expands cluster from them.
It works well, if clusters have a similar density.

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
model = DBSCAN()
l_pred = model.fit_predict(data)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
data_cluster = data[l_pred != -1]         # Selecting only points that belong to a cluster
label_cluster = l_pred[l_pred != -1]      # Selecting only the labels (clusters) that are actual clusters
data_outlier = data[l_pred == -1]         # Selecting the outliers (not part of any cluster)
data_outlier.plot.scatter("x", "y", ax=ax, c="k", alpha=.5)
data_cluster.plot.scatter("x", "y", ax=ax, c=label_cluster, cmap="rainbow", colorbar=False)

With the `eps` hyperparameter we can define the distance between to points in a points neighborhood.
So, lower values mean smaller neighborhoods.
The default is `.5`, thus we start with a smaller value.

In [ ]:
multi_plot(DBSCAN(), data, "eps", [.4,.3,.2,.175,.15,.125,.1,.05])

As we can see, since the circles and curves have a higher density, they are not detected when the neighborhood is to big.
Only in the last three plots, we see that they get detected.
And then, the other cluster are just counted as noise.

## Exercises

### Ex01 - Silhouette Analysis

In this exercise, you are going to run a silhouette analysis for a given sample dataset.
We also provide you a helper method that you can use.

In [ ]:
from sklearn.base import clone
from matplotlib import cm

def silhouettes(data, model, n_clusters):
    m = clone(model)                                 # Create copy of the model
    m.set_params(n_clusters=n_clusters)              # Setting the numbers of clusters
    l_pred = m.fit_predict(data)
    coef = silhouette_score(data, l_pred)            # Get the silhouette coefficient
    print(f"Silhouette coefficient for {n_clusters}: {np.round(coef, 6)}")
    values = silhouette_samples(data, l_pred)        # Get the silhouette values
    colors = cm.get_cmap("rainbow", n_clusters)      # Getting colors
    fig, ax = plt.subplots(figsize=(10,10))
    fig.suptitle(f"Silhouette Analysis with {n_clusters} Clusters")
    margin = 10
    y_bottom = margin
    # Plotting the silhouette values
    for c in range(n_clusters):
        values_c = values[l_pred == c]               # Just getting values of cluster c
        values_c = np.sort(values_c)                 # Sorting the values
        size_c = len(values_c)                       # Getting the number of values
        y_top = y_bottom + size_c                    # Calculating the value on the y-axis to draw to
        color = colors(c/n_clusters)                 # Selecting the color of the cluster
        # Plotting the values
        (ax.fill_betweenx(                           # Fill an area
            np.arange(y_bottom, y_top),              # Range on y-axis to fill
            0,                                       # Start value on x-axis
            values_c,                                # End value on x-axis
            fc=color, ec=color, alpha=.7))           # Some styling
        # Plotting the cluster number to the left
        ax.text(-.05, y_bottom + .5 * size_c, str(c))
        y_bottom = y_top + margin                    # Calculating the next start value on the y-axis
    ax.axvline(coef, c="k", ls="--")              # Plotting a vertical line for the silhouette coefficient
    ax.set(title="Silhouette Values", xlabel="Silhouette Values", ylabel="Clusters", yticks=[], xlim=(-.1,1))

Now load the dataset from **Ex13_01_Data.csv**.

Create a new dataset without the `label` column.

Let's assume we only have 2 clusters in the data.
Run a k-Means clustering algorithm and calculate the silhouette coefficient.

What's the silhouette coefficient when assuming 5 clusters?

Calculate the average silhouette value of each of those 5 clusters.

Calculate the silhouette coefficients for k-Means with 2 - 10 clusters.

Use the helper method `silhouettes(data, model, n_clusters)` to plot clusters 2 - 10.

What's the correct number of clusters?
Run a cluster analysis with the number of clusters you think are in the dataset and create a scatter plot.

Congratulations!
You run a simple silhouette analysis.

#### Solution

In [ ]:
# %load ./Ex13_01_Sol.py

### Ex02 - Density Plots

In this exercise, you are going to use some advanced plotting methods.
To use them, you need some data.
Thus, load **Ex13_02_Data.csv**.

You should already know this dataset.
It's the same you used in the last exercise for preprocessing.

To get a broad overview how the data is distributed, let's do a `pairplot()`.

As you can see, some data is correlated.
Maybe if we throw in some color, we get a better idea.
Use the `cylinders` for coloring the plots.

Now, use the `kdeplot()` method and show the density when comparing `horsepower` and `mpg`.

Use the `kdeplot()` method again, but this time for `acceleration` and `weight`.
And plot the points as well.
Use the `cylinders` for coloring.

Use the `jointplot()` method to shoe a comparison of `weight` and `acceleration`.

Do the same, but this time use  `kind=kde`.

Use the `jointplot()` method with `kind=hex` to show a comparison between `displacement` and `horsepower`.

Congratulations!
You now know some more awesome methods to plot visualize your data.

#### Solutions

In [ ]:
# %load ./Ex13_02_Sol.py

### Ex03 - DBSCAN

In this exercise, you run a DBSCAN algorithm for a given dataset.
Load **Ex13_03_Data.csv**.

Run the DBSCAN algorithm and predict the clusters.

Plot the dataset with the clusters.

As you can see, all the outliers have a color.
Plot the clusters, but this time the outliers should be plotted in black.

As you can see, the inner and outer circles are always considered as one cluster.
Reduce the `eps` to `.1` to get better results and plot the clusters again.

Congratulations!
You've successfully applied the DBSCAN clustering algorithm.

#### Solution

In [ ]:
# %load ./Ex13_03_Sol.py